# 文書分類ハンズオン

## 機械学習モデル開発のワークフローと本章で扱う内容

なお、動作確認は以下の環境で行いました。

- Ubuntu 16.04
- Python 3.7
- PyTorch 1.3

## 文書分類ハンズオン

### 本章で扱うツールの解説

#### Hugging Face

### データセットの準備

#### livedoor ニュースコーパス

今回は日本語における自然言語処理の試験用データセットとしてしばしば用いられる「livedoor ニュースコーパス」を用います。

livedoorニュースはもともと株式会社ライブドアが運営するニュースサイトでしたが、株式会社ライブドアが旧ハンゲームジャパン株式会社であるNHN Japan株式会社に買収され、現在はNHN Japanが社名変更したLINE株式会社により運営されています。livedoorニュースの記事の一部には「クリエイティブ・コモンズライセンス『表示 – 改変禁止』」が適用されており、営利目的を含めて再配布可能となっています。該当するニュース記事を2012年9月上旬に株式会社ロンウイットが収集し、HTMLタグの除去などクリーニングを施した状態で公開しているのが「livedoor　ニュースコーパス」です。

livedoor ニュースコーパスは以下のリンクよりダウンロード可能です。

https://www.rondhuit.com/download.html#ldcc

オープンソースの全文検索システムApache Solrで扱いやすいようXML形式でニュースが格納されている `livedoor-news-data.tar.gz` と、シンプルに各々のニュースをテキストファイルとして扱っている　`ldcc-20140209.tar.gz` が公開されています。

今回は後者の `ldcc-20140209.tar.gz` をダウンロードしてください。`tar xzvf ldcc-20140209.tar.gz` などにより解凍すると `text` という名前のディレクトリが出てきます。`text` ディレクトリの中身の構造は以下の通りです。

```
text
├── CHANGES.txt
├── README.txt
├── dokujo-tsushin
├── it-life-hack
├── kaden-channel
├── livedoor-homme
├── movie-enter
├── peachy
├── smax
├── sports-watch
└── topic-news
```

`dokujo-tsushin` から `topic-news` はディレクトリであり、それぞれにニュース記事のテキストが格納されています。

```
text
├── CHANGES.txt
├── README.txt
├── dokujo-tsushin
│   ├── LICENSE.txt
│   ├── dokujo-tsushin-4778030.txt
│   ├── dokujo-tsushin-4778031.txt
│   ├── dokujo-tsushin-4782522.txt
...
```

ニュース提供元は以下の9つです。記事の本文だけを見て、その記事がどのカテゴリに属しているのか（独女通信のニュースなのか、ITライフハックのニュースなのか、など）を判別する文書分類モデルを作成するのが本章の目的です。

- 独女通信 (http://dokujo.jp/)
- ITライフハック (http://news.livedoor.com/category/vender/223/)
- 家電チャンネル (http://news.livedoor.com/category/vender/kadench/)
- livedoor HOMME (http://news.livedoor.com/category/vender/homme/)
- MOVIE ENTER (http://news.livedoor.com/category/vender/movie_enter/)
- Peachy (http://news.livedoor.com/category/vender/ldgirls/)
- エスマックス (http://news.livedoor.com/category/vender/smax/)
- Sports Watch (http://news.livedoor.com/category/vender/208/)
- トピックニュース (http://news.livedoor.com/category/vender/news/)

#### ラベリング

### 前処理

#### 形態素解析
#### ストップワード除去

### 文書分類モデル

### 評価と比較

### モデルのデプロイ

## まとめ

In [1]:
import torch